In [1]:
from google.colab import drive
drive.mount('/content/drive')  #listになかったのでドライブ上にデータ保存し、ドライブをマウント。

Mounted at /content/drive


In [2]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.6 MB/s eta 0:00:00


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/signate/モジュール系/EDA')
sys.path.append('/content/drive/MyDrive/signate/モジュール系/Feature Engineering')
sys.path.append('/content/drive/MyDrive/signate/モジュール系/meanF1')

In [4]:
from obj_to_cat import to_cat_data
from reduce_mem import reduce_mem_usage

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.model_selection import KFold, StratifiedKFold

from catboost import CatBoostClassifier, Pool, metrics, cv

前処理

In [6]:
feat = "feat00"
train = pd.read_csv(f"/content/drive/MyDrive/signate/債務不履行/途中経過_csv/feat_train_{feat}.csv")
test = pd.read_csv(f"/content/drive/MyDrive/signate/債務不履行/途中経過_csv/feat_test_{feat}.csv")
train_origin = pd.read_csv("/content/drive/MyDrive/signate/債務不履行/データセット/train.csv")

# オーディナルカテゴリ特徴量のカラム
#------------------------------------------------------------------------------
## カテゴリ変数の抽出
cols_notcat = ['MIS_Status', 'Term', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob',
               'DisbursementGross', 'GrAppv', 'SBA_Appv','population', 'density',
               'lr_avg_ppi','ApprovalToDisbursement_days','outlier'] # 数値特徴量 'DisbursementGross_minus_SBA_Appv'
# county_col = ['county_gdp','sector_count']
time_col = ["DisbursementDate_timestamp",	"ApprovalDate_timestamp",	"LoanEnd_timestamp"]
dissba = ['DisbursementGross_plus_SBA_Appv','DisbursementGross_minus_SBA_Appv', 'DisbursementGross_times_SBA_Appv','DisbursementGross_divided_by_SBA_Appv']
sbagr = ['SBA_Appv_plus_GrAppv','SBA_Appv_minus_GrAppv', 'SBA_Appv_times_GrAppv','SBA_Appv_divided_by_GrAppv']
disgr = ['DisbursementGross_plus_GrAppv','DisbursementGross_minus_GrAppv', 'DisbursementGross_times_GrAppv','DisbursementGross_divided_by_GrAppv']
# year_col = ["USREC_SUM"]
# urb_col = ["HOUPCT_RUR","ALAND_PCT_RUR","RUR_pct"]
city_col = ["city_Term_mean","city_NoEmp_mean"]

cols_notcat = cols_notcat + city_col + dissba + sbagr + disgr + time_col # 数値特徴量

#-------------------------------------------------------------------------------
# 全カテゴリ特徴量
cols_cat = [c for c in test.columns if c not in cols_notcat]
# ターゲットエンコ変数
target_encoded_cols = [c for c in test.columns if "target_" in c]
# 0rdinal cat変数
cols_cat = [c for c in cols_cat if c not in target_encoded_cols]
#-------------------------------------------------------------------------------

### カテゴリ特徴量の欠損を-1で埋める（Catboostはカテゴリ特徴量の欠損を受け付けない）
for col in cols_cat:
  train[col] = train[col].fillna(-1)
  test[col] = test[col].fillna(-1)

### カテゴリカル特徴量をcategory型に変換
for col in cols_cat:
    train[col] = train[col].astype('category')
    test[col] = test[col].astype('category')

In [7]:
### 多重共線性の削除
numeric_cols = train.select_dtypes(include=['number']).columns
# 数値型カラム間の相関行列を計算
corr_matrix = train[numeric_cols].corr()
# 相関係数が0.95を超えるカラムペアを探索
# 相関行列は対称なので、対角線より上（または下）の要素のみを検討すれば十分
high_corr_pairs = []
del_col = []

for i, col1 in enumerate(corr_matrix.columns):
    for j, col2 in enumerate(corr_matrix.index):
        if i < j:  # 対角線より上の要素のみをチェック
            if abs(corr_matrix[col1][col2]) > 0.95:  # 絶対値を取ることで正負の相関も考慮
                high_corr_pairs.append((col1, col2, corr_matrix[col1][col2]))

# 相関係数が0.95を超えるカラムペアの表示
for pair in high_corr_pairs:
    del_col.append(pair[0])
    print(f"{pair[0]} と {pair[1]} の相関係数: {pair[2]:.2f}")

DisbursementGross と GrAppv の相関係数: 0.97
DisbursementGross と DisbursementGross_plus_SBA_Appv の相関係数: 0.99
DisbursementGross と SBA_Appv_plus_GrAppv の相関係数: 0.97
DisbursementGross と DisbursementGross_plus_GrAppv の相関係数: 0.99
GrAppv と SBA_Appv の相関係数: 0.98
GrAppv と DisbursementGross_plus_SBA_Appv の相関係数: 0.99
GrAppv と SBA_Appv_plus_GrAppv の相関係数: 1.00
GrAppv と DisbursementGross_plus_GrAppv の相関係数: 0.99
SBA_Appv と DisbursementGross_plus_SBA_Appv の相関係数: 0.98
SBA_Appv と SBA_Appv_plus_GrAppv の相関係数: 0.99
SBA_Appv と DisbursementGross_plus_GrAppv の相関係数: 0.97
DisbursementGross_plus_SBA_Appv と SBA_Appv_plus_GrAppv の相関係数: 0.99
DisbursementGross_plus_SBA_Appv と DisbursementGross_plus_GrAppv の相関係数: 1.00
DisbursementGross_times_SBA_Appv と SBA_Appv_times_GrAppv の相関係数: 0.99
DisbursementGross_times_SBA_Appv と DisbursementGross_times_GrAppv の相関係数: 0.98
DisbursementGross_divided_by_SBA_Appv と DisbursementGross_divided_by_GrAppv の相関係数: 0.96
SBA_Appv_plus_GrAppv と DisbursementGross_plus_GrAppv の相関係数: 0.99
SBA_Appv_ti

In [8]:
del_col = list(set(del_col))

train = train.drop(del_col, axis=1)
test = test.drop(del_col, axis=1)

In [9]:
### 'MIS_Status'を一番右に移動
# まず'MIS_Status'を除いたカラムリストを作成
cols = [col for col in train.columns if col != 'MIS_Status']
# 次に'MIS_Status'をカラムリストの最後に追加
cols.append('MIS_Status')
# 新しいカラム順序でDataFrameを再構成
train = train[cols]


# 説明変数のカラム
cols_exp = test.columns
# ターゲット変数
col_target = "MIS_Status"

実行

In [10]:
import hyperopt
from hyperopt import hp
from catboost import CatBoostClassifier, cv, Pool
import numpy as np

def tune_catboost_params(train, cols_exp, col_target, cols_cat):
    x = train[cols_exp]           # npに変換しない
    y = train[col_target]

    params_space = {
        'learning_rate': hyperopt.hp.uniform('learning_rate', 0.01, 0.3),
        'l2_leaf_reg': hyperopt.hp.randint('l2_leaf_reg', 1, 10),
        'depth': hyperopt.hp.randint('depth', 4, 10),
        'bagging_temperature': hyperopt.hp.uniform('bagging_temperature', 0.0, 1.0),
        'random_strength': hyperopt.hp.uniform('random_strength', 1, 20)
    }
    trials = hyperopt.Trials()

    def hyperopt_objective(params):
        params_add = {
            'loss_function': "Logloss",  # 二値分類の損失関数
            'eval_metric': "AUC",  # 評価指標をAUCに設定
            'od_type': 'Iter',  # early stopping
            'od_wait': 25,  # early stopping
            'logging_level': 'Silent',
            'use_best_model': True,
            'random_seed': 42  # 再現性のためのシード
        }
        params.update(params_add)

        # NumPy配列を使用してPoolオブジェクトを初期化
        train_pool = Pool(data=x, label=y, cat_features=cols_cat)

        cv_data = cv(
            train_pool,
            params,
            fold_count=5,  # 5-fold cross-validation
            logging_level='Silent',
            stratified=True,  # 分類問題での層化サンプリング
            shuffle=True  # データのシャッフル
        )
        max_auc = np.max(cv_data['test-AUC-mean'])  # AUCの最大値を目的関数として使用

        return -max_auc  # hyperoptは最小化を行うため、AUCの最大化のために負の値を使用

    # パラメータ最適化の実行
    params_tuned = hyperopt.fmin(
        hyperopt_objective,
        space=params_space,
        algo=hyperopt.tpe.suggest,
        max_evals=25,
        trials=trials,
    )

    return params_tuned


In [11]:
def train_catboost(train, cols_exp, col_target, cols_cat, params=None):
    # cols_expにおけるカテゴリ変数のインデックス (Catboostモデリングに使用)
    cols_cat_idxs = [i for i, c in enumerate(cols_exp) if c in cols_cat]

    if params is None:
        params = {}

    # 二値分類用のパラメータ設定
    params_add = {'loss_function': "Logloss",  # 二値分類用
                  'od_type': 'Iter',  # early stopping
                  'od_wait': 50,  # early stopping
                  'logging_level': 'Silent',
                  'use_best_model': True,
                  'random_state': 42  # 結果の再現性のため
                 }
    params.update(params_add)

    x = train[cols_exp]
    y = train[col_target]

    # 層化K-fold
    skf = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
    y_valid_pred_lst = []
    idx_valid_lst = []
    clf_lst = []

    # cross validation
    for fold, (idx_train, idx_valid) in enumerate(skf.split(x, y)):
        print("fold", fold)
        x_train = x.iloc[idx_train]     #idx_trainの説明変数
        x_valid = x.iloc[idx_valid]     #idx_validの説明変数
        y_train = y.iloc[idx_train]     #idx_trainの目的変数
        y_valid = y.iloc[idx_valid]     #idx_validの目的変数

        # catboost modeling
        clf = CatBoostClassifier(**params)
        clf.fit(
            x_train, y_train,
            cat_features=cols_cat_idxs,
            eval_set=(x_valid, y_valid),
            plot=True
        )

        # OOF予測
        y_valid_pred = clf.predict_proba(x_valid)
        y_valid_pred_lst.append(y_valid_pred)
        idx_valid_lst.append(idx_valid)
        clf_lst.append(clf)

    # OOF予測の統合
    idx_valid = np.hstack(idx_valid_lst)
    y_valid_pred = np.vstack(y_valid_pred_lst)
    oof_pred = y_valid_pred[np.argsort(idx_valid)]

    return clf_lst, oof_pred

In [12]:
def predict_test(x_test, clf_lst):
    y_test_pred_lst = []

    for clf in clf_lst:
        y_test_pred = clf.predict_proba(x_test)
        y_test_pred_lst.append(y_test_pred)

    y_test_pred = np.mean(y_test_pred_lst, axis=0)
    return y_test_pred

In [13]:

# hyperoptを用いたcatboostパラメータ最適化
#params_tuned = tune_catboost_params(train, cols_exp, col_target, cols_cat)

params_tuned = {'bagging_temperature': 0.9573725082971074,
 'depth': 5,
 'l2_leaf_reg': 9,
 'learning_rate': 0.11337151684849904,
 'random_strength': 5.491985711362313,
 'loss_function': 'Logloss',
 'od_type': 'Iter',
 'od_wait': 50,
 'logging_level': 'Silent',
 'use_best_model': True,
 'random_state': 42}

# train Catboost model
clf_lst, oof_pred = train_catboost(train, cols_exp, col_target, cols_cat, params_tuned)


# predict test with CV ensemble
y_test_pred = predict_test(test[cols_exp], clf_lst)

# record
oof_pred_df = pd.DataFrame(oof_pred, columns=[f"MIS_Status_{h}" for h in range(2)])
test_pred_df = pd.DataFrame(y_test_pred, columns=[f"MIS_Status_{h}" for h in range(2)])


fold 0


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

fold 1


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

fold 2


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

fold 3


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

fold 4


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [14]:
oof_pred_df

,MIS_Status_0,MIS_Status_1
0,0.022885,0.977115
1,0.037070,0.962930
2,0.030085,0.969915
3,0.030598,0.969402
4,0.059883,0.940117
...,...,...
42302,0.020615,0.979385
42303,0.129937,0.870063
42304,0.050689,0.949311
42305,0.025767,0.974233


In [15]:
from sklearn.metrics import f1_score

# 実際のターゲット値
y_true = train[col_target].values

# OOF予測値をクラスに変換（確率から最も高いクラスを選択）
# 二値分類の場合、0.5を閾値としてクラスを決定することが一般的です
y_oof_pred_class = (oof_pred[:, 1] > 0.5).astype(int)

# mean F1スコア（マクロ平均）を計算
mean_f1_score_macro = f1_score(y_true, y_oof_pred_class, average='macro')
print(f'Mean F1 Score (Macro): {mean_f1_score_macro}')

Mean F1 Score (Macro): 0.6449457660074802


In [16]:
# save
oof_pred_df.to_csv(f"/content/drive/MyDrive/signate/債務不履行/model_feat/oof_pred_catboost_{feat}.csv")
test_pred_df.to_csv(f"/content/drive/MyDrive/signate/債務不履行/model_feat/test_pred_catboost_{feat}.csv")

In [17]:
params_tuned

{'bagging_temperature': 0.9573725082971074,
 'depth': 5,
 'l2_leaf_reg': 9,
 'learning_rate': 0.11337151684849904,
 'random_strength': 5.491985711362313,
 'loss_function': 'Logloss',
 'od_type': 'Iter',
 'od_wait': 50,
 'logging_level': 'Silent',
 'use_best_model': True,
 'random_state': 42}

In [18]:
"""
0.6504
{'bagging_temperature': 0.8613235727733222,
 'depth': 9,
 'l2_leaf_reg': 2,
 'learning_rate': 0.03738982944556876,
 'random_strength': 12.58511646150734,
 'loss_function': 'Logloss',
 'od_type': 'Iter',
 'od_wait': 50,
 'logging_level': 'Silent',
 'use_best_model': True,
 'random_state': 42}

0.6489
 {'bagging_temperature': 0.4301091961539275,
 'depth': 9,
 'l2_leaf_reg': 8,
 'learning_rate': 0.05536861934416303,
 'random_strength': 11.477210411321622,
 'loss_function': 'Logloss',
 'od_type': 'Iter',
 'od_wait': 50,
 'logging_level': 'Silent',
 'use_best_model': True,
 'random_state': 42}
 """

"\n0.6504\n{'bagging_temperature': 0.8613235727733222,\n 'depth': 9,\n 'l2_leaf_reg': 2,\n 'learning_rate': 0.03738982944556876,\n 'random_strength': 12.58511646150734,\n 'loss_function': 'Logloss',\n 'od_type': 'Iter',\n 'od_wait': 50,\n 'logging_level': 'Silent',\n 'use_best_model': True,\n 'random_state': 42}\n\n0.6489\n {'bagging_temperature': 0.4301091961539275,\n 'depth': 9,\n 'l2_leaf_reg': 8,\n 'learning_rate': 0.05536861934416303,\n 'random_strength': 11.477210411321622,\n 'loss_function': 'Logloss',\n 'od_type': 'Iter',\n 'od_wait': 50,\n 'logging_level': 'Silent',\n 'use_best_model': True,\n 'random_state': 42}\n "